In [1]:
from langchain.agents import Tool
from langchain.utilities import GoogleSearchAPIWrapper
from langchain.tools.file_management.write import WriteFileTool
from langchain.tools.file_management.read import ReadFileTool

In [ ]:
from dotenv import load_dotenv
import os

load_dotenv('C:/Users/dell/OneDrive/Documents/LangChain and Vector database/.env')

google_api_key = os.getenv("GOOGLE_API_KEY_1")
search_engine_id = os.getenv("GOOGLE_CSE_ID")

os.environ["GOOGLE_API_KEY"] = google_api_key

In [2]:
search = GoogleSearchAPIWrapper()

tools = [
    Tool(
        name='search',
        func=search.run,
        description='Useful for when you need to answer questions about current events. You should ask targeted questions',
        return_direct=True,
    ),
    WriteFileTool(),
    ReadFileTool()
]

### Agent Memory Setup

In [12]:
from langchain.vectorstores import FAISS
from langchain.docstore import InMemoryDocstore
from langchain_google_genai.embeddings import GoogleGenerativeAIEmbeddings

embedding = GoogleGenerativeAIEmbeddings(model='models/embedding-001')

embedding_size = 768

In [13]:
import faiss
index = faiss.IndexFlatL2(embedding_size)
vectorstore = FAISS(embedding.embed_query, index, InMemoryDocstore({}), {})

`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.


In [18]:
from langchain_experimental.autonomous_agents import AutoGPT
from langchain.llms import HuggingFaceHub

agent = AutoGPT.from_llm_and_tools(
    ai_name="Bale",
    ai_role='assistant',
    tools=tools,
    llm=HuggingFaceHub(repo_id='google/flan-t5-large',
    model_kwargs={'temperature':0}),
    memory=vectorstore.as_retriever(),
)

agent.chain.verbose = True

In [ ]:
task = "Provide an analysis of the major historical events that led to the French Revolution"

agent.run([task])